<a href="https://colab.research.google.com/github/shunrei9841-sudo/Guadalupe/blob/main/Parcial%202-3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import math

# Parámetro de la distribución exponencial
lambda_param = 3
N = 1000000  # Número de simulaciones

# --- A) Simulación Simple (Monte Carlo Directo) ---
print("--- A) Simulación Simple ---")

# Generar N variables uniformes U ~ Unif(0, 1)
U = np.random.uniform(0, 1, N)

# Generar N variables X ~ Exp(lambda=3) usando el método de la inversa
# X = -1/lambda * ln(1 - U)
X = -1/lambda_param * np.log(1 - U)

# Función de interés: g(X) = sqrt(X)
g_X = np.sqrt(X)

# Estimación del valor esperado E[sqrt(X)]
E_sqrt_X_simple = np.mean(g_X)

# Varianza de la estimación (V[g(X)] / N)
# Primero calculamos la varianza de g(X)
Var_g_X_simple = np.var(g_X, ddof=1) # ddof=1 para estimación muestral
Var_E_simple = Var_g_X_simple / N

print(f"Estimación E[sqrt(X)] (Simple): {E_sqrt_X_simple:.6f}")
print(f"Varianza de g(X) (Simple): {Var_g_X_simple:.8f}")
print(f"Varianza de la Estimación (Simple): {Var_E_simple:.10f}")
print("-" * 30)


# --- B) Variables Antitéticas ---
print("--- B) Variables Antitéticas ---")

# Las variables antitéticas se basan en U y U_anti = 1 - U
U_anti = 1 - U

# Generar X_anti a partir de U_anti
X_anti = -1/lambda_param * np.log(1 - U_anti) # X_anti = -1/lambda * ln(U)

# Función de interés para la variable antitética
g_X_anti = np.sqrt(X_anti)

# Estimador de variables antitéticas: g_anti = (g(X) + g(X_anti)) / 2
g_anti = (g_X + g_X_anti) / 2

# Estimación del valor esperado E[sqrt(X)] usando variables antitéticas
E_sqrt_X_anti = np.mean(g_anti)

# Varianza del estimador antitético
Var_g_anti = np.var(g_anti, ddof=1)
Var_E_anti = Var_g_anti / N

# Porcentaje de reducción de varianza
Reduccion_anti = (Var_g_X_simple - Var_g_anti) / Var_g_X_simple * 100

print(f"Estimación E[sqrt(X)] (Antitéticas): {E_sqrt_X_anti:.6f}")
print(f"Varianza del Estimador g_anti: {Var_g_anti:.8f}")
print(f"Varianza de la Estimación (Antitéticas): {Var_E_anti:.10f}")
print(f"Reducción de Varianza (g(X) vs g_anti): {Reduccion_anti:.2f}%")
print("-" * 30)


# --- C) Variable de Control ---
print("--- C) Variable de Control ---")

# Variable de control: c(X) = X. Sabemos analíticamente E[X] = 1/lambda
E_X_analitico = 1/lambda_param

# El estimador de la variable de control es:
# g_ctrl = g(X) - b * (X - E[X])
# El óptimo b* es: b* = Cov[g(X), X] / Var[X]

# 1. Calcular Cov[g(X), X]
# NumPy calcula la matriz de covarianza, donde Cov[g(X), X] es el elemento [0, 1] o [1, 0]
Cov_matrix = np.cov(g_X, X)
Cov_gX_X = Cov_matrix[0, 1]

# 2. Calcular Var[X]
# Para Exp(lambda=3), Var[X] = 1 / lambda^2. Verificamos con la muestra.
Var_X_sample = np.var(X, ddof=1) # Usamos la varianza muestral de X
# Var_X_analitico = 1 / lambda_param**2 # Para el cálculo analítico b_analítico

# 3. Calcular el factor b* óptimo (usando valores muestrales)
b_optimo = Cov_gX_X / Var_X_sample

# 4. Calcular el estimador g_ctrl
g_ctrl = g_X - b_optimo * (X - E_X_analitico)

# Estimación del valor esperado E[sqrt(X)] usando variable de control
E_sqrt_X_ctrl = np.mean(g_ctrl)

# Varianza del estimador de variable de control
Var_g_ctrl = np.var(g_ctrl, ddof=1)
Var_E_ctrl = Var_g_ctrl / N

# Porcentaje de reducción de varianza
Reduccion_ctrl = (Var_g_X_simple - Var_g_ctrl) / Var_g_X_simple * 100

print(f"Estimación E[sqrt(X)] (Control): {E_sqrt_X_ctrl:.6f}")
print(f"Valor óptimo b* (Muestra): {b_optimo:.6f}")
print(f"Varianza del Estimador g_ctrl: {Var_g_ctrl:.8f}")
print(f"Varianza de la Estimación (Control): {Var_E_ctrl:.10f}")
print(f"Reducción de Varianza (g(X) vs g_ctrl): {Reduccion_ctrl:.2f}%")


# --- C) Verificación Analítica de la Reducción de Varianza ---
print("\n--- C) Verificación Analítica (Reducción de Varianza) ---")

# La reducción de varianza teórica (relación de varianza) es:
# Var[g_ctrl] / Var[g(X)] = 1 - (Corr[g(X), X])^2
# Donde Corr[g(X), X] = Cov[g(X), X] / (sqrt(Var[g(X)]) * sqrt(Var[X]))

# 1. Calcular el coeficiente de correlación de la muestra (Rho)
Rho_gX_X = np.corrcoef(g_X, X)[0, 1]

# 2. Calcular la relación de varianza teórica basada en la muestra
Relacion_Varianza_Teorica = 1 - Rho_gX_X**2

# 3. Calcular la reducción de varianza teórica
Reduccion_Analitica = (1 - Relacion_Varianza_Teorica) * 100

print(f"Coeficiente de Correlación Rho[sqrt(X), X] (Muestra): {Rho_gX_X:.6f}")
print(f"Relación de Varianza Teórica (1 - Rho^2): {Relacion_Varianza_Teorica:.6f}")
print(f"Reducción de Varianza Teórica Esperada: {Reduccion_Analitica:.2f}%")
print(f"Reducción Observada (Simulación): {Reduccion_ctrl:.2f}% (Se aproxima a la teórica)")

--- A) Simulación Simple ---
Estimación E[sqrt(X)] (Simple): 0.511727
Varianza de g(X) (Simple): 0.07159679
Varianza de la Estimación (Simple): 0.0000000716
------------------------------
--- B) Variables Antitéticas ---
Estimación E[sqrt(X)] (Antitéticas): 0.511698
Varianza del Estimador g_anti: 0.00189326
Varianza de la Estimación (Antitéticas): 0.0000000019
Reducción de Varianza (g(X) vs g_anti): 97.36%
------------------------------
--- C) Variable de Control ---
Estimación E[sqrt(X)] (Control): 0.511629
Valor óptimo b* (Muestra): 0.767514
Varianza del Estimador g_ctrl: 0.00609522
Varianza de la Estimación (Control): 0.0000000061
Reducción de Varianza (g(X) vs g_ctrl): 91.49%

--- C) Verificación Analítica (Reducción de Varianza) ---
Coeficiente de Correlación Rho[sqrt(X), X] (Muestra): 0.956487
Relación de Varianza Teórica (1 - Rho^2): 0.085133
Reducción de Varianza Teórica Esperada: 91.49%
Reducción Observada (Simulación): 91.49% (Se aproxima a la teórica)
